In [2]:
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import model_selection

In [3]:
df = pd.read_csv('data/train.csv')
df = df.dropna()
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,total_acc,initial_list_status,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,record_id
3,4200.0,36 months,13.33,142.19,C,C3,Receptionist,< 1 year,MORTGAGE,21600.0,...,19.0,w,0.0,1.0,INDIVIDUAL,0.0,165.0,28187.0,14500.0,453447199
7,14000.0,36 months,15.31,487.45,C,C2,Department of the Army,10+ years,MORTGAGE,96000.0,...,29.0,f,0.0,1.0,INDIVIDUAL,0.0,0.0,276267.0,39700.0,453322484
12,20000.0,36 months,11.99,664.20,B,B3,"Sr. Specialist, Tax",10+ years,RENT,110000.0,...,35.0,f,0.0,1.0,INDIVIDUAL,0.0,0.0,41806.0,54600.0,453423047
16,8650.0,36 months,12.12,287.80,B,B3,Haven Hospice,8 years,RENT,76000.0,...,20.0,f,0.0,1.0,INDIVIDUAL,0.0,0.0,614403.0,10000.0,453286318
17,25000.0,60 months,17.27,624.95,C,C5,Cheney Public Schools,6 years,MORTGAGE,108000.0,...,40.0,f,0.0,1.0,INDIVIDUAL,0.0,0.0,337866.0,23800.0,453277374


In [4]:
data = df.drop(['emp_title', 'home_ownership', 'pymnt_plan','record_id', 'earliest_cr_line', 'grade', 'sub_grade', 'application_type', 'issue_d', 'zip_code', 'policy_code', 'verification_status', 'purpose', 'initial_list_status', 'addr_state','emp_length','term'], axis=1)

data['loan_amnt'] = data['loan_amnt'].astype(float)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69010 entries, 3 to 200188
Data columns (total 19 columns):
loan_amnt                     69010 non-null float64
int_rate                      69010 non-null float64
installment                   69010 non-null float64
annual_inc                    69010 non-null float64
loan_status                   69010 non-null int64
dti                           69010 non-null float64
delinq_2yrs                   69010 non-null float64
inq_last_6mths                69010 non-null float64
mths_since_last_delinq        69010 non-null float64
open_acc                      69010 non-null float64
pub_rec                       69010 non-null float64
revol_bal                     69010 non-null float64
revol_util                    69010 non-null float64
total_acc                     69010 non-null float64
collections_12_mths_ex_med    69010 non-null float64
acc_now_delinq                69010 non-null float64
tot_coll_amt                  69010 non-null

In [6]:
array = data.values
Y = data['loan_status'].values

X = data.drop(['loan_status'], axis=1)

from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=0)
X, Y = ros.fit_resample(X, Y)

x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.3, random_state=42)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [8]:
train = list(zip(x_train,y_train))
test = list(zip(x_test, y_test))

In [9]:
import math

def get_distance(data1, data2):
    points = zip(data1, data2)
    diffs_squared_distance = [pow(a - b, 2) for (a, b) in points]
    return math.sqrt(sum(diffs_squared_distance))

In [10]:
from operator import itemgetter
 
def get_neighbours(training_set, test_instance, k):
    distances = [_get_tuple_distance(training_instance, test_instance) for training_instance in training_set]
 
    sorted_distances = sorted(distances, key=itemgetter(1))
 
    sorted_training_instances = [tuple[0] for tuple in sorted_distances]
 
    return sorted_training_instances[:k]
 
def _get_tuple_distance(training_instance, test_instance):
    return (training_instance, get_distance(test_instance, training_instance[0]))

In [11]:
from collections import Counter
 
def get_majority_vote(neighbours):
    classes = [neighbour[1] for neighbour in neighbours]
    count = Counter(classes)
    return count.most_common()[0][0]

In [13]:
predictions = [] 
k = 3
for x in range(len(x_test)):    
    neighbours = get_neighbours(training_set=train, test_instance=test[x][0], k=3)
    majority_vote = get_majority_vote(neighbours)
    predictions.append(majority_vote)     
print ('\nThe overall accuracy of the model is: ' + str(accuracy_score(y_test, predictions)) + "\n")
print("Precision:",metrics.precision_score(y_test, predictions))
print("Recall:",metrics.recall_score(y_test, predictions))


The overall accuracy of the model is: 0.5652173913043478

Precision: 0.5591376966401242
Recall: 0.5829115205507188


In [14]:
print(metrics.roc_auc_score(y_test, predictions))

0.5653601427454789
